In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.15.0'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
## Load dataset
data = pd.read_csv(r"C:\Users\amita\Desktop\Krish Naik\GEN AI\Mine\DL NLP\ANNClassification\Churn_Modelling.csv")

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
## Preprocess data
## Drop irrelevent features
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
## Geography and gender we need to convert from categorical to continous
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
#One hot encode geography
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_encoded = ohe.fit_transform(data[['Geography']])
geo_encoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [11]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
## Combine all cols
data=pd.concat([data.drop('Geography', axis=1),geo_encoded_df], axis=1)

In [15]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [16]:
## Pickle dump
#saving encoders
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(ohe, file)


In [17]:
## Divide dataset into independent and dependent features
X = data.drop(['Exited'], axis=1)
y = data['Exited']

In [18]:
## Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
## scaling
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [20]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [21]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scalar,file)

## ANN

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [25]:
(X_train.shape[1],)

(12,)

In [29]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #first hidden layer connected with input layer
    Dense(32, activation='relu'), # hidden layer 2
    Dense(1, activation='sigmoid') # Output layer
])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [32]:
## FOrward and backward compile model
#model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"]) directly can be used like this or from above cell
model.compile(optimizer=opt, loss="binary_crossentropy",metrics=["accuracy"])

In [33]:
## Setup tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [34]:
## Setup Early stopping
# ES when we run for 100 epochs but at 20 epoch model is trained as best model so its not required to train fir full 100 epochs
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [35]:
## training the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 0.3958 - accuracy: 0.8355 - val_loss: 0.3487 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3565 - accuracy: 0.8544 - val_loss: 0.3442 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3470 - accuracy: 0.8601 - val_loss: 0.3451 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3453 - accuracy: 0.8591 - val_loss: 0.3500 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3413 - accuracy: 0.8610 - val_loss: 0.3436 - val_accuracy: 0.8635
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3371 - accuracy: 0.8610 - val_loss: 0.3467 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3369 - accuracy: 0.8633 - val_loss: 0.3466 - val_accuracy: 0.85

In [36]:
model.save('model.h5')

C:\Users\amita\anaconda\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
## Load tensorboard extension
%load_ext tensorboard

In [39]:
%tensorboard --log_dir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "C:\Users\amita\anaconda\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\amita\anaconda\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\amita\anaconda\Scripts\tensorboard.exe\__main__.py", line 4, in <module>
  File "C:\Users\amita\anaconda\lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\amita\anaconda\lib\site-packages\tensorboard\default.py", line 30, in <module>
    import pkg_resources
  File "C:\Users\amita\anaconda\lib\site-packages\pkg_resources\__init__.py", line 95, in <module>
    from jaraco.text import drop_comment, join_continuation, yield_lines
  File "C:\Users\amita\anaconda\lib\site-packages\setuptools\_vendor\jaraco\text\__init__.py", line 12, in <module>
    from jaraco.context import Exception